## MANB2163 - Social Network Analytic
## Mohd Nazri Nawi
## Assignment No 1 - Part 2


##### Data is obtain from Stanford University SNAP Data Set. Source citation:
##### S. Kumar, F. Spezzano, V.S. Subrahmanian, C. Faloutsos. Edge Weight Prediction in Weighted Signed Networks. IEEE International Conference on Data Mining (ICDM), 2016.
##### S. Kumar, B. Hooi, D. Makhija, M. Kumar, V.S. Subrahmanian, C. Faloutsos. REV2: Fraudulent User Prediction in Rating Platforms. 11th ACM International Conference on Web Searchand Data Mining (WSDM), 2018.


In [ ]:
install.packages("igraph")
install.packages("dplyr")
install.packages("ape")
install.packages("fields")
library(fields)
library(ape)
library(dplyr)
library(igraph) 

# Plotting Network using igraph

### 1. Explain the data fields. Choose the specific data fields that you want to use to build the network. 
 
- Dataset: soc-sign-bitcoinotc.csv
- Description: Bitcoin OTC web of trust network
- Where:
      Source - Node of ID source (rater)
      Target - Node of ID target (ratee)
      Rating - The source's rating for the target, ranging from -10 to +10 in steps of 1
      Time   - The time of the rating, measured as seconds since Epoch.

- This is a directed graph network
- For this assignment, we only interested in the source(V1), target (V2) dataset.
- Rating (V3) will be set at 10,which means that at rating 10 the buyer trust the seller as he trust himself. This is usually true for close friends and associates they know in person.


### 2. Read the network data from csv files and display the data in a table in R.

In [ ]:
# We only interested with rating = 10, in order to limit no of nodes to cut down processing time.
dat = read.csv('soc-sign-bitcoinotc.csv', header=FALSE)
dat = filter (dat, V3 == 10)
mat = as.matrix(dat)
g <- graph.data.frame(dat)


 ### 3. Plot the network using igraph. 

In [ ]:
# Plot of network
plot(g,layout=layout_nicely,vertex.label=NA, 
       vertex.color="gold", vertex.size = 5, edge.arrow.size = 0.1, edge.width=2.0, edge.color = "blue")

 ### 4. Examine the data: 
- Find number of nodes 
- Find number of edges 
- Find the edgelist (“from”, “to”)

In [ ]:
# As graph node
g

In [ ]:
# Vertices
# From result below, vertices = 5,881
V(g)

In [ ]:
# Edge
# From result below, Edge = 35,592
E(g)

In [ ]:
# Edgelist
el <- as_edgelist(g, names=T)
head(el)
tail(el)
graph_from_edgelist(el)

### 5: Simplify the network by removing all the multiple edges and loops.

In [ ]:
# Simplified Network
g1 = simplify(g,remove.multiple = TRUE,remove.loops = TRUE,edge.attr.comb = igraph_opt("edge.attr.comb"))

In [ ]:
# Verification of simplified network
is_simple(g1)

### 6. Extract the adjacency matrix of the network and display.

In [ ]:
# Adjacency matrix of the netwrk and display
g2 <- as_adjacency_matrix(g1)
g3 <- graph.adjacency(g2)
g3
V(g3)
E(g3)

In [ ]:
# Plot of simplified network, g3
plot(g3,layout=layout_with_fr,vertex.label=NA, vertex.color=rgb(.25, .5, .3, alpha=.5), vertex.size = 5, 
     edge.arrow.size = 0.1, edge.width=2.0, edge.color = "gray40", edge.curved = 1.0)

### 7. Replace the vertex labels (auto label) of each node with the node names stored in the data table.

In [ ]:
# Replace the vertex label with the node names stored in data table

plot(g3,layout=layout_nicely,vertex.label=V(g3)$name, main = "Network plot with node number label", 
       vertex.color="gold", vertex.size = 5, edge.arrow.size = 0.5, edge.width=2.0, edge.color = "gray40")

### 8. Generate node color based on the type of nodes. Add legends to explain the meaning of colors.

In [ ]:
# Node color based on the type of node. In this case, we will choose Community detection based on label propagation:
clu <- components(g3)

#Sample function on vertices

color_num = 150:1

#create a color palette of the same size as the number of vertices.
jet.colors <- colorRampPalette( rainbow( length( unique(color_num) ) ) )
color_spectrum <- jet.colors( length( unique(color_num ) ) )

# Map the pallete to the order of values on vertices
ordered <- order(color_num)
color <- vector(length = length(ordered),mode="double")
for ( i in 1:length(ordered) )
{
    color[ ordered[i] ] <- color_spectrum [ i ]
}
clu$membership = color

plot(g3, vertex.color=color, vertex.size = 5, edge.arrow.size = 0.5, 
     edge.width=2.0, edge.color = "gray40",main = "Network with color of component membership", vertex.label = NA) 
image.plot(legend.only=T, zlim=range(color_num), col=color_spectrum )

### 9. Set the node size based on the degree of the node.

In [ ]:
# Set the node size based on the degree of the node.

deg <- degree(g3, mode="all")
plot(g3, layout=layout_nicely, vertex.size = deg, edge.arrow.size = .2, main = "Network plot based on node degree", 
     vertex.color = rgb(.25, .5, .3, alpha=.5), vertex.label=NA, , edge.width=2.0, edge.color = "gray40")

### 10. Use 5 different type of layouts in igraph to plot the network and display it. Explain the changes in the network structure and compare your network based on the different layout algorithm that you have used.

These plots are based on Force-directed graph drawing algorithms i.e Kamada Kawai and Fruchterman Reingold etc. The purpose is to position the nodes of a graph in 2D or 3D space so that all the edges are in equal length and minimal crossing edges.

In [ ]:
# Fruchterman Reingold
plot(g3, layout=layout_with_fr, vertex.size = deg, edge.arrow.size = .2, main = "Fruchterman Reingold", 
     vertex.color = rainbow(100), vertex.label=NA, , edge.width=2.0, edge.color = "gray40")

In [ ]:
# Kamada Kawai
plot(g3, layout=layout_with_kk, vertex.size = deg, edge.arrow.size = .2, main = "Kamada Kawai", 
     vertex.color = rainbow(100), vertex.label=NA, , edge.width=2.0, edge.color = "gray")

In [ ]:
# Star
plot(g3, layout= layout_as_star, vertex.size = deg, edge.arrow.size = .2, main = " Layout: Star", 
     vertex.color = rainbow(100), vertex.label=NA, , edge.width=0.1, edge.color = "gray")

In [ ]:
# Nicely
plot(g3, layout=layout_nicely, vertex.size = deg, edge.arrow.size = .2, main = "Nicely", 
     vertex.color = rainbow(100), vertex.label=NA, , edge.width=0.5, edge.color = "gray")

In [ ]:
# Random
plot(g3, layout=layout_randomly, vertex.size = deg, edge.arrow.size = .2, main = "Random", 
     vertex.color = rainbow(100), vertex.label=NA, , edge.width=0.1, edge.color = "gray")

### 11. Give a degree range for the nodes you want to explore and keep only the nodes that ties between the chosen range. Display the network. Display the evolution of network by changing the node degree range till you get a network with separated components.

In [ ]:
deg = degree(g3)
degSorted <-sort.int(deg,decreasing=T,index.return=FALSE)
df = data.frame(degSorted)
head(df)
tail(df)

In [ ]:
degree_freq <- data.frame(table(deg))
head(degree_freq)
tail(degree_freq)

In [ ]:
n1 <- delete.vertices(g3, V(g3)[ degree(g3) >3 ])
plot(n1, layout=layout_with_kk, main = "Network Evolution. Degrees: n = 1", vertex.color=rainbow(degree(n1)*10), 
     vertex.size=degree(n1), edge.arrow.size= 0.3, edge.width=1, vertex.label = NA)

In [ ]:
n5 <- delete.vertices(g3, V(g3)[ degree(g3) > 5 ])
plot(n5, layout=layout_with_kk, main = "Network Evolution. Degrees: n < 5", vertex.color=rainbow(10), 
     vertex.size=degree(n5), edge.width=1.0, edge.arrow.size= 0.3, vertex.label = NA)

In [ ]:
n12 <- delete.vertices(g3, V(g3)[ degree(g3) > 12 ])
n = n12
plot(n, layout=layout_with_kk, main = "Network Evolution. Degrees: n = 35", vertex.color=rainbow(10), 
     vertex.size=degree(n), edge.width=0.3, edge.arrow.size= 0.2, vertex.label = NA)

In [ ]:
n30 <- delete.vertices(g3, V(g3)[ degree(g3) > 30 ])
n = n30
plot(n, layout=layout_with_kk, main = "Network Evolution. Degrees: n < 30", vertex.color=rainbow(10), 
     vertex.size=degree(n), edge.arrow.size= 1, vertex.label = NA)

# Network and node descriptions

### 1. Using the same network (simplified network), find the density of the network.

In [ ]:
n <- g3
cat("Density of Network:", edge_density(n, loops = FALSE))

In [ ]:
#for an undirected network
ecount(n)/(vcount(n12)*(vcount(n)-1))*2 

### 2. Find the number of triangles formed in the network.

In [ ]:
tri=cliques(n,min=3,max=3)
cat("No of Triangles:", nrow(matrix(tri)))

### 3. What is the diameter of the network?

In [ ]:
cat("Diameter of Network:", diameter(n))

### 4. The nodes color that pass through the longest shortest path.

In [ ]:
diam <- get_diameter(g3, directed=T)
diam

In [ ]:
n=g3
vcol <- rep("light blue", vcount(n))
vcol[diam] <- "gold"
ecol <- rep("gray80", ecount(n))
ecol[E(n, path=diam)] <- "red" 

# E(net, path=diam) finds edges along a path, here 'diam'

plot(n, vertex.size=6, vertex.color=vcol, 
     edge.width=5, edge.color=ecol, edge.arrow.mode=0.2, vertex.label=NA, )

### 5. List the degree of nodes and display it in a table. Create a histogram of the node degree.

In [ ]:
df = data.frame(deg)
head(df)
tail(df)

In [ ]:
hist(deg)

In [ ]:
deg.dist <- degree_distribution(g3, cumulative=T, mode="all")
plot( x=0:max(deg), y=1-deg.dist, pch=19, cex=1.2, col="orange", 
     xlab="Degree", ylab="Cumulative Frequency")

# Centrality Values

### 1. Rank the nodes based on degree, betweenness, closeness and eigenvector centrality value and display it in a table.

In [ ]:
# Rank by Degree
deg <- degree(g3)
degSorted <-sort.int(deg,decreasing=T,index.return=FALSE)
df <- data.frame(degSorted)
head(df)
tail(df)

In [ ]:
# Rank by betweeness
btw <- betweenness(g3,  v = V(g3),  directed = TRUE,  weights = NULL,  nobigint = TRUE,  normalized = FALSE)
btw <-data.frame(sort.int(btw,decreasing=T,index.return=FALSE))
head(btw)
tail(btw)

In [ ]:
# Rank by closeness
close <- closeness(g3,  vids = V(g3),  mode = c("all"),  weights = NULL,  normalized = FALSE)
clo <-data.frame(sort.int(close,decreasing=T,index.return=FALSE))
head(clo)
tail(clo)

In [ ]:
#Rank by Eigen Centrality
ec <-eigen_centrality(g3,directed = TRUE,scale = TRUE,weights = NULL,options = arpack_defaults)
ec <-data.frame(sort.int(ec$vector,decreasing=T,index.return=FALSE))
head(ec)
tail(ec)

### 2. Find the nodes with highest degree, betweenness centrality, closeness and eigenvector centrality values.

From the calculated result above:
- Highest degree: **35** at **Node 25**
- Highest betweeness centrality: **654.83** at **Node 905**
- Highest closeness: **3.956260e-06** at **Node 25**
- Highest eigenven vector centrality: **1.000** at **Node 2962**

### 3. Find the hubs in the network and display each of it in a different color.

In [ ]:
hs <- hub_score(g3, weights=NA)$vector
plot(g3, vertex.size=hs*10, main="Hubs", edge.arrow.size= 0.3, vertex.label=NA, edge.width=2)

# Distance and Path

### 1. Calculate the average path length for both (undirected and directed network).

In [ ]:
path1 <- mean_distance(g3, directed = TRUE, unconnected = TRUE)
path2 <- mean_distance(g3, directed = FALSE, unconnected = TRUE)
cat(" Average Directed Path:", path1)
cat(" Average Undrected Path:", path2)

### 2. Using the undirected network, find all the shortest paths from one node to another and the length of all shortest paths in the graph.

In [ ]:
asp <-all_shortest_paths(
  g3,
  from = "25",
  to = V(g3),
  mode = c("all"),
  weights = NULL
)

summary(asp)

### 3. Find the shortest path from the node with highest betweenness centrality (broker) to all other nodes. Color the path that has the longest shortest path from the broker to its destination node. Repeat the same for nodes with highest degree and eigenvector centrality values.

In [ ]:
plist <- do.call(c,lapply(V(g3), function(v) get.shortest.paths(g,v,V(g), output='epath')$epath))
psize <- data.frame(i = 1:length(plist), plength = sapply(plist,length))
top10 <- head(psize[order(-psize$plength),],10)
elist <- unlist(plist[top10$i])
finalg <- subgraph.edges(g, elist)

In [ ]:
longest.shortest.paths <- function(graph){
    # Return edgelist of all node-pairs between which the shortest path
    # in a graph are the longest shortest path observed in that graph.

    # Get all the shortest paths of a graph
    shortest.paths = shortest.paths(graph)

    # Make sure that there are no Inf-values caused by isolates in the graph
    shortest.paths[shortest.paths == Inf] <- 0

    # What nodes in the distance matrix are linked by longest shortest paths?
    el <- which(shortest.paths==max(shortest.paths), arr.ind=TRUE)
    colnames(el) <- c("i","j")
    (el)
}

longest.shortest.paths(g3)

### 4. Identify the immediate neighbours of the node with highest degree centrality value. Set colors to plot the neighbours. Display the network and explain the neighbours with this important node.

In [ ]:
neigh.nodes <- neighbors(g3, V(g3), mode="all")

# Set colors to plot the neighbors:

vcol[neigh.nodes] <- "red"
plot(g3, vertex.size = 5, vertex.label=NA, vertex.color=vcol, edge.arrow.size= 0.3,edge.color="gray40", edge.width=2)

### 5. Identify the immediate neighbours of the node with highest eigenvector centrality value. Set colors to plot the neighbours. Display the network and explain the neighbours with this important node.

In [ ]:
V(g3)$ec <- eigen_centrality(g, directed=T, weights=NA)$vector
normalize <- function(x){(x-min(x))/(max(x)-min(x))}
(V(g3)$ec_index <- round(normalize(V(g3)$ec) * 9) + 1)
V(g3)$color <- colorRampPalette(c("turquoise", "yellow","red"))(10)[V(g3)$ec_index]
table(V(g3)$color)
plot(g3, vertex.label=NA, vertex.size=5)

# Subgroups and Communities

### 1. Find cliques in the network and display it. How many cliques that you can find in the network?

In [ ]:
g3.sym <- as.undirected(g3, mode= "collapse", edge.attr.comb=list(weight="sum", "ignore"))
c = cliques(g3.sym) # list of cliques       
s = sapply(cliques(g3.sym), length) # clique sizes
l = largest_cliques(g3.sym) # cliques with max number of nodes
vcol <- rep("gray", vcount(g3.sym))
vcol[unlist(largest_cliques(g3.sym))] <- "red"


In [ ]:
plot(as.undirected(g3.sym), vertex.size = 5, vertex.label=NA, vertex.color=vcol, edge.color="gray20", edge.width=2)

### 2. Find a community detection algorithm in igraph. Explain how it works. Apply the community detection on your network and display the network. Each community must be in its own color.
- Find the number of communities that occur.
- Find its membership
- Find how modular the graph partitioning is. (High modularity for a partitioning reflects dense connections within communities and sparse connections across communities)

In [ ]:
eb <- cluster_edge_betweenness(as.undirected(g3))
dendPlot(eb, mode="hclust", main = "denplot")

In [ ]:
plot(eb, as.undirected(g3), vertex.label=NA, vertex.size=5,edge.arrow.mode=0.2, edge.color="blue", edge.width=3)

### b. Membership

In [ ]:
comm <- membership(eb)
summary(comm)
modularity(eb)

### c. Modular the graph partitioning

In [ ]:
modularity(eb)

### Community detection algorithm in igraph

In [ ]:
clp <- cluster_label_prop(g3)
plot(clp, as.undirected(g3), vertex.label=NA, vertex.size=5,edge.arrow.mode=0.2, edge.color="blue", edge.width=3)

In [ ]:
cfg <- cluster_fast_greedy(as.undirected(g3))
plot(cfg, as.undirected(g3), vertex.label=NA, vertex.size=5,edge.arrow.mode=0.2, edge.color="blue", edge.width=3)

In [ ]:
V(g3)$community <- cfg$membership
colrs <- adjustcolor( c("gray50", "tomato", "gold", "yellowgreen"), alpha=.8)
plot(g3, vertex.color=colrs[V(g3)$community], vertex.label=NA, vertex.size=5,edge.arrow.mode=0.2, edge.color="blue", edge.width=3)